Rusty Bargain used car sales service is developing an app to attract new customers. In that app, you can quickly find out the market value of your car. You have access to historical data: technical specifications, trim versions, and prices. You need to build the model to determine the value. 

Rusty Bargain is interested in:

- the quality of the prediction;
- the speed of the prediction;
- the time required for training

## Data preparation

In [1]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error

In [2]:
print("Loading data...")
data = pd.read_csv('/datasets/car_data.csv')

data = data.drop(columns=['NumberOfPictures', 'PostalCode'])

categorical_features = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']
for col in categorical_features:
    data[col] = data[col].fillna('unknown')

data = data[(data['RegistrationYear'] >= 1900) & (data['RegistrationYear'] <= 2023)]

data = data.drop_duplicates()

target = data['Price']
features = data.drop(columns=['Price'])

features_full_train, features_test, target_full_train, target_test = train_test_split(
    features, target, test_size=0.2, random_state=12345)

features_train, features_valid, target_train, target_valid = train_test_split(
    features_full_train, target_full_train, test_size=0.25, random_state=12345)

ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)
ohe.fit(features_train[categorical_features])

def encode(df):
    encoded = pd.DataFrame(ohe.transform(df[categorical_features]), index=df.index)
    encoded.columns = ohe.get_feature_names(categorical_features)


Loading data...
